Should the piece rotation methods live on the piece object?
Or stay on the piece manager?

## Testing Piece Insertions Into Game And Board

In [ ]:
from blokusduo.game import BlokusDuoGame, BlokusDuoBoard, Action
from pathlib import Path

p = Path("blokusduo") / "pieces.json"
game = BlokusDuoGame(p)
board = game.initialise_board()

In [ ]:
for p in board.piece_manager.pieces.values():
    print(p)
    break

In [ ]:
from blokusduo.pieces import BidirectionalDict

d = BidirectionalDict()

d[a1] = 14
d.items()

In [ ]:
board.as_array, board._black_player.potential_placement_points

In [ ]:
a1

d = {}

d.update({a1: 14})

In [ ]:
a1 = Action(14, "Identity", 9, 4)
board.insert_piece(a1, -1)

In [ ]:
board.as_array, board._black_player.potential_placement_points

In [ ]:
a2 = Action(1, "Identity", 8, 5)
board.insert_piece(a2, -1)

In [ ]:
board.as_array, board._black_player.potential_placement_points

In [ ]:
for k, _ in board._black_player.potential_placement_points.items():
    board.as_array[k] = -2
board.as_array

In [ ]:
board._white_player.potential_placement_points

In [ ]:
placement_cache = {
    (8,3): {
        
    }
}

Niavely, we iterate over every single possible placement point.
We iterate over every single piece and every single orientation of this piece
Iterate again over all the possible x,y points that constitute inserting this piece into that point.
Takes ages.

Need functionality for efficiently generating all possible legal moves for player

ALGORITHM:
- If no corners/game just started -> return cached data we are passed on init
- If corner present but empty, we run through all REMAINING piece orientations, calculate all possible insertion indicies -> put into the cache
- If corner present, return cached value and check values not altered by last placement
- We can remove last piece id inserted since this cache was checked
- Run quick check remaining insertions to see if there are all valid
- NB: We don't need to run the corner check on the cached actions because we know, by definition they must have at least one corner.

Faster way of checking corners?
Make dictionary bi directional/have two dictionaries. Populate one with action and other with locations of 1 and locations of sides after action
When insert piece, search hashmap for 1 location
If piece same player, search for intersections of side too
If either of these conditions are violated, remove action

LIVES ON PLAYER
placement_point -> piece_id, orientation, l_idx, w_idx -> piece_id AKA have first dict bidirectional so we can quickly remove values of last insertion piece id
                -> piece_id -> action

coordinates_of_sides -> list[action] } For each insertion 1, check if index in here and remove corresponding actions

coordinates_of_1s -> list[action]    } When we insert a piece, we check this dict to see if the piece insertion physically blocks piece placement

VALID MOVES ALGORITHM

- Game just started (do not just check if there are no corners) -> return cached data passed to board on init
- After one inserted piece, found corners but corners are empty, populate with all possible actions
- Store locations of 1s for each action, store locations of sides for each action (and a way of mapping these to actions)

When a piece is inserted
- Remove piece orientations with that piece id
- Remove placement points the piece is on
- Remove piece orientations that are blocked by this piece
- Remove piece orientations blocked by same side piece next door


DATA STRUCTURE IDEAS

Idea 1: Have a couple of flat dictionaries with helper classes for pointing to the right places
- One Dictionary: action_indx -> action VALUES OF THIS ARE WHAT GETS RETURNED
- Dictionary: placement_point -> action_indx & List of placement_point
- Dictionary: 1 position -> list of action idx
- Dictionary: Side position -> list of action idx

NB: Thing holding actions needs to be able to remove actions for piece id, remove actions for placement points removed, remove actions for 1 violated and for side violation 

NB: 
- Sometimes two different placement points can have the same action so you need to account for this
- 

Alternatively

Could just keep a list of placement points into actions around, and loop through all actions to check if they're still valid
Only have to check if sides or ones of action are invalid after piece placed because you know there must be a corner present

Idea 2: Do things more naively as benchmark figure
- Dictionary: placement point -> piece_id -> action

Now you can easily whip through placement points to remove them, or remove piece ids and also you can produce all actions

NB: Can still have actions repeated so makesure to call set on the result

In [ ]:
from blokusduo.pieces import pieces_loader, Piece, Orientation, PieceManager

placements = {
    (8, 3): {},
    (8, 5): {}
}

In [ ]:
board._white_player.potential_placement_points

In [ ]:
board._white_player.pieces_remaining

In [ ]:
for point in board._black_player.potential_placement_points:
    print(point)

In [ ]:
board._white_player.potential_placement_points

In [ ]:
game.piece_manager.pieces[11].identity

In [ ]:
action_2 = Action(14, "Identity", 9, 4)

In [ ]:
action = Action(11, "Identity", 4, 4)

In [ ]:
board.insert_piece(action, 1)

In [ ]:
board.insert_piece(action_2, -1)

In [ ]:
piece_orientation = board.piece_manager.get_piece_orientation_array(4, "Rot90")

In [ ]:
board._board_representation

In [ ]:
game.piece_manager.pieces[14].identity

In [ ]:
game.initial_actions[-1][:10]

In [ ]:
for a in game.initial_actions[-1][:10]:
    board.insert_piece(a, -1)
    print(board._board_representation)
    print("\n")
    board = game.initialise_board()

In [ ]:
for a in game.initial_actions[1][:10]:
    board.insert_piece(a, 1)
    print(board._board_representation)
    print("\n")
    board = game.initialise_board()

In [ ]:
list(*(res[1][0]))

In [ ]:
for i, action in enumerate(res[1]):
    print(action)
    board.insert_piece()
    if i >10:
        break

In [ ]:
board.board_representation

In [ ]:
%%time
res = bdg._calculate_and_cache_initial_moves()

In [ ]:
len(res[-1])

In [ ]:
len(res[1])

In [ ]:
len(bdg.initial_moves[1])

In [ ]:
for k,v in piece_manager._piece_orientation_lookup.items():
    print(k, v.basis)
    break

In [ ]:
piece_id, piece in self._piece_orientation_lookup.items():

In [ ]:
%%time
from pathlib import Path
from blokusduo.pieces import Piece, Orientation, pieces_loader
import json
from dataclass_wizard import fromdict
import numpy as np
from blokusduo.game import CoordinateIndexDecoder

f = Path("blokusduo") / "pieces.json"
piece_manager = pieces_loader(f)

In [ ]:
# For each piece, get all the basis_orientations. 
# For each piece-bo combo, see how many different x, y coordinates there are for the top left corner
# Such that there is a 1 on the start piece.

In [ ]:
n=14
board = np.zeros(n ** 2, dtype=int).reshape(n, n)

black_start = (4, 4)
white_start = (9, 9)
board[white_start] = 1
board[black_start] = -1
board

In [ ]:
def try_fit_onto_start_corner(piece_id, piece, board, start) -> tuple[int, Orientation, int, int]:
    # Think about this as iterating over the piece, moving the board the other way
    # Want to return actions
    length, width = piece.shape
    for i in range(length):
        for j in range(width):
            if piece[(i, j)] ==1:
                yield (start[0] - i, start[1] - j) 
            

In [ ]:
piece_manager.pieces[14]

In [ ]:
ans = []

def gen_func():
    for i in range(3):
        yield i

ans+= gen_func()
ans += gen_func()

ans

In [ ]:
ans =[]
for k, v in piece_manager.pieces.items():
    for o in v.basis_orientations:
        ans+=try_fit_onto_start_corner(k, v, board, white_start)

ans

In [ ]:
for thing in piece_manager.all_basis_orientations_arrays_for_piece(21):
    print(thing)

In [ ]:
for k, v in piece_manager.pieces.items():
    print(k, v)
    break

In [ ]:
n=14
board = np.zeros(n ** 2, dtype=int).reshape(n, n)

black_start = (4, 4)
white_start = (9, 9)
board[white_start] = 1
board[black_start] = -1
board

In [ ]:
piece.basis_orientations

In [ ]:
piece = piece_manager.pieces[4]

In [ ]:
np.sum([ v.identity.sum() for k, v  in piece_manager.pieces.items()])

In [ ]:


action = [14, "Identity", 9, 4]

length_idx = n-1 - action[3]
width_idx = action[2]

piece_shape = piece_manager.get_piece_with_orientation(action[0], action[1])
piece

In [ ]:
length, width = piece_shape.shape
for i in range(length):
    for j in range(width):
        board[length_idx + i, width_idx + j] = piece_shape[i, j]

board

In [ ]:
board

In [ ]:
import numpy as np
n = 14

x, y = 4, 5


board[(4, 4)] = -1
board[(9, 9)] = 1

board

In [ ]:
pi = np.random.rand(
    14*14*91 + 1
)

moves = pi[:-1]

moves.reshape((14, 14, 91))[:, :, 3].shape

In [ ]:
type([:, :, 3])

In [ ]:
action = 1, 3, 39

type(action)

In [ ]:
pieces[0].identity.dtype

In [ ]:
len(pieces)

In [ ]:
from collections import deque

d = deque()

for p in pieces:
    d.append(p)


In [ ]:
len(d)

In [ ]:
d.popleft()

In [ ]:
import numpy as np

bored = np.array([
    [0, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 0],
]
)

In [ ]:
white_pieces = np.array([[1, 1, 1, 1]]).reshape(-1, 1)
black_pieces = -1*np.array([[1, 1, 1, 1]]).reshape(-1, 1)

In [ ]:
np.zeros(2 * n).reshape(n, 2).dtype

In [ ]:
import numpy as np

n = 14

np.zeros(2 * n).astype(int).reshape(n, 2).dtype

In [ ]:
res = np.concatenate((white_pieces, bored), axis=1)

In [ ]:
res

In [ ]:
res.size

In [ ]:
res[res == -1].size

In [ ]:
pieces[:10]

In [ ]:
np.concatenate((res, black_pieces), axis=1)

In [ ]:
white_pieces.reshape(-1, 1)

In [ ]:
bored

In [ ]:
np.rot90(np.flip(bored, 1), 3)

In [ ]:
pieces[20].identity

In [ ]:
pieces[20].rot90

In [ ]:
pieces[0].identity.tobytes()

In [ ]:
from main import args

In [ ]:
# How to change args
from main import args
from dataclass_wizard import fromdict, asdict
from core.config import RunConfig

# d = asdict(args)
# d["runName"] = "full_run_because_why_not_1"
# new_args = fromdict(RunConfig, d)

In [ ]:
action_size = 14, 14, 91
action_size 

In [ ]:
14*14*91 + 1, 14*14*92

In [ ]:
import math

math.prod(action_size)

## Check Nets

In [ ]:
from core.config import NetConfig
NetConfig?

In [ ]:
from blokusduo.neuralnets.net import AlphaBlokusDuo
from tictactoe.neuralnets.net import AlphaTicTacToe
from tictactoe.game import TicTacToeGame as game
from core.config import NetConfig
import torch 
from torchviz import make_dot

cfg = NetConfig(0.001,0.3,10,32,True,512, 19)
g = game(3)

bd_net = AlphaBlokusDuo(cfg)
ttt_net = AlphaTicTacToe(g, cfg)

In [ ]:
sum(p.numel() for p in bd_net.parameters())/1e6 

In [ ]:
sum(p.numel() for p in ttt_net.parameters())/1e6 

In [ ]:
bd_net.parameters

In [ ]:
bd_input = torch.rand(2*14*18).reshape(-1, 14, 18)
ttt_input = torch.rand(2*3*3).reshape(-1, 3, 3)

In [ ]:
bd_res = bd_net(bd_input)
ttt_res = ttt_net(ttt_input)

In [ ]:
make_dot(ttt_res, params=dict(list(ttt_net.named_parameters()))).render("ttt_torchviz", format="png")
make_dot(bd_res, params=dict(list(bd_net.named_parameters()))).render("bd_torchviz", format="png")

In [ ]:
len(ttt_res[0]), len(bd_res[0])

In [ ]:
ttt_res[0].shape

In [ ]:
bd_res[1].shape, ttt_res[1].shape

In [ ]:
import os

os.environ["PATH"]

In [ ]:
res[0].shape

In [ ]:
14*14*95

In [ ]:
torch.prod(torch.tensor(action_size))

In [ ]:
import torch 

torch.rand(torch.prod(action_size)).shape

In [ ]:
import numpy as np

output = np.random.rand(np.prod(action_size))
output.reshape(action_size).shape

In [ ]:
16*16 -14*14

In [ ]:
s = 0
for p in pieces:
    s += p.symmetry
s

In [ ]:
len(pieces)

In [ ]:
pieces

# Plot data

In [ ]:
import plotly.io as pio
from pickle import Pickler, Unpickler
from pathlib import Path
import os 
import pandas as pd
import plotly_express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt

pio.renderers.default = 'jupyterlab'

## Report Generation

## Read data

In [ ]:
%%time
import os

# Training data
data_root = args.training_data_directory

run_completed = False

if not run_completed:
    filenames = [f for f in os.listdir(data_root) if "train" in f]  
    
    print(filenames)
    
    def loader(data_root: Path, filename: str):
        with open(data_root / filename, "rb") as f: 
            data = Unpickler(f).load()
        return pd.DataFrame(data)
    
    
    ds = [loader(data_root, f) for f in filenames]
    df = pd.concat(ds, copy=False).assign(average_loss = lambda df: df.average_pi_loss + df.average_v_loss)
else:
    df = pd.read_parquet(data_root / "data.parquet")

In [ ]:
# Arena data
ad = pd.read_parquet(args.arena_data_directory)

In [ ]:
# Timings

td = pd.read_parquet(args.timings_directory)

In [ ]:
td[td.cycle_stage == "WholeCycle"].time_elapsed.sum() / 60 

In [ ]:
td.reset_index().apply(lambda df: type(df))

In [ ]:
td.columns

In [ ]:
td.reset_index().apply(lambda df: df[df.cycle_stage == "WholeCycle"])

In [ ]:
from pandas import DataFrame

def make_loss_curves(df: DataFrame) -> tuple[go.Figure]:
    
    df_plot = df.sort_values(["generation", "epoch", "batch_number"]).astype({"epoch": "category", "generation": "category"})
    agg_df_plot = df_plot.groupby("generation").agg({"average_loss": "last"}).reset_index()
    
    fig1 = px.line(
        agg_df_plot, x="generation", y="average_loss" 
    )

    fig2 = px.scatter(
        df_plot, x="batch_number", y="average_loss", color="epoch", hover_data =["generation"],
        labels={"x": "Batch Number", "y": "Average Loss"}
    )

    fig1.update_layout(
        width=1200, 
        height=600,
        xaxis_title="Generation",
        yaxis_title="Average Loss", 
        title="Average Total Pi And V Loss per Generation",
    )

    fig2.update_traces(marker_size=2)
    fig2.update_layout(
        width=1200,
        height=600, 
        yaxis_title="Average Loss",
        xaxis_title="Batch Number",
        title="Average Total Pi And V Loss As A Function Of Batch Number For Each Generation And Epoch",
    )
    
    return fig1, fig2

def make_arena_data_plot(arena_data: pd.DataFrame) -> go.Figure:
    arena_data = arena_data.assign(**{
        "Percentage Losses": lambda df: 100*df.losses/ (df.wins + df.losses + df.draws),
        "Percentage Wins": lambda df: 100*df.wins/ (df.wins + df.losses + df.draws)
    }).sort_values(["generation"]).astype({"generation": "category"})
    
    fig = px.bar(
        arena_data, x= "generation", y=["Percentage Wins", "Percentage Losses"], labels={"x": "Generation", "y": "Percentage Value"}
    )

    fig.update_layout(
        width=1200,
        height=600, 
        yaxis_title="Percentage Value",
        xaxis_title="Generation",
        title="Win & Loss Rate Of New Net Against Predecessor",
    )
    fig.update_yaxes(range=[0, 100])
    return fig


def make_performance_statistics_plot(timings_data: pd.DataFrame) -> go.Figure:
    cycles = df.cycle_stage.unique()
    fig = make_subplots(cols=len(cycles), shared_yaxes=True, subplot_titles=cycles, x_title="Generation", y_title="Time Elapsed / S")
    
    for i, cycle in enumerate(cycles):
        filt = lambda df: df.cycle_stage == cycle
        fig.add_trace(
            go.Bar(
                x= df[filt]["generation"], y = df[filt]["time_elapsed"], name=cycle
            ), row=1, col=i+1
        )
        
    
    fig.update_layout(height=600, width=1200,
                      title_text=f"Performance Statistics For run")
    
    return fig

In [ ]:
args.training_data_directory

In [ ]:
args.run_name

In [ ]:
def create_html_report(args):
    loss_data = pd.read_parquet(args.training_data_directory / "data.parquet")
    arena_data = pd.read_parquet(args.arena_data_directory)
    timings_data = pd.read_parquet(args.timings_directory)

    fig1, fig2 = make_loss_curves(loss_data)
    fig3 = make_arena_data_plot(arena_data)
    fig4 = make_performance_statistics_plot(timings_data)

    with open('tmp.html', 'w') as f:
        f.write(fr"""
            <html><head><title> Output Data in an HTML file \ 
            </title></head><body><h1> <u>AlphaBlokus</u> Report </h1> 
            <h2>For run: '{args.run_name}' </h2> </body></html>"""
               ) 
        f.write(fig1.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write(fig2.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write(fig3.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write(fig4.to_html(full_html=False, include_plotlyjs='cdn'))



In [ ]:
%%time

create_html_report(args)

## Performance Statistics

In [ ]:
root = args.timings_directory
df = pd.read_parquet(root / "timings.parquet")


In [ ]:
make_performance_statistics_plot(df)

In [ ]:
print(
    f"Total time elapsed: {df[df.cycle_stage == "WholeCycle"].time_elapsed.sum()/60} Minutes!"
)

## Window Size

In [ ]:
import numpy as np
def _generation_window_size(max_value, generation: int) -> int:
    # Dynamically adjusted window size for generation lookback
    # To get window size, I've fit the line:
    # y = a*x + b to the points (x=0, y=5) and (x=2*max_window_size, y=max_window_size)
    # This allows a good combination of forgetting earlier games but retaining good memory as iterations go on

    if generation <= 5:
        return 5
    max_val = max_value

    if 2 * max_val > generation:
        return round(generation * (max_val - 5) / (2 * max_val) + 5)

    return max_val

x = np.arange(30)
y = [_generation_window_size(15, i) for i in x]

px.scatter(x=x, y=y)

##  Playing The Net

In [ ]:
%%time
from core.coach import Coach
from pathlib import Path
from core.config import RunConfig, NetConfig, MCTSConfig
from tictactoe.tictactoegame import TicTacToeGame as Game
from tictactoe.neuralnets.wrapper import NNetWrapper
from main import args
from core.mcts import MCTS

import numpy as np

g = Game(3)
nnet = NNetWrapper(g, args)
c = Coach(g, nnet, args)

# Play a game

In [ ]:
# Load in bot to play
nnet.load_checkpoint(filename='best.pth.tar')
computer = MCTS(g, nnet, args.mcts_config)
computer_player = lambda x: np.argmax(computer.get_action_prob(x, temp=0))

In [ ]:
# Create human player interface

class HumanTicTacToePlayer:
    def __init__(self, game):
        self.game = game

    def play(self, board):
        display(board)
        valid = self.game.get_valid_moves(board, 1)
        print (f"Valid moves: {np.where(valid==1)}")
        
        while True:
            input_move = input()
            a = int(input_move)
            if a > -1 and a < 11:
                return a
            else:
                raise ValueError(f"Must be integer in range (0, 10), {a} is out of range")
        return a

human = HumanTicTacToePlayer(g)

In [ ]:
from core.arena import Arena
from IPython.display import display

arena = Arena(human.play, computer_player, g, display=display)

In [ ]:
# Run this cell to play against the computer

arena.play_game(verbose=True)

### Try and read in examples

In [ ]:
from pickle import Pickler, Unpickler
from pathlib import Path
import os

In [ ]:
%%time
with open(args.self_play_history_directory / "self_play_history_29.pickle", "rb") as f:
    examples = Unpickler(f).load()

In [ ]:
import pandas as pd
from pathlib import Path

p = Path(r'C:\Users\Henry\PycharmProjects\Alpha\temp\test_run\TrainingData')

def loader(filepath: Path):
    with open(filepath, "rb") as f:
        data = Unpickler(f).load()
    return pd.DataFrame(data)

df = loader(p / "train_1.data")

In [ ]:
df.pi_loss.astype(float)

In [ ]:
pd.read_parquet(p / "data.parquet")